## Hartree Fock en el átomo de Helio


In [1]:
import numpy as np



En este laboratorio vamos a implementar una rutina que permita obtener
la energía hartree-fock del átomo de helio a través de las ecuaciones
de Roothaan. Para eso vamos a expresar el orbital 1s del helio en terminos
de dos funciones bases:

\begin{equation}
\phi_1(r) = 2\alpha_1^{3/2}e^{-\alpha_1 r}Y^0_0, \qquad \phi_2(r) = 2\alpha_2^{3/2}*e^{-\alpha_2 r}Y^0_0,
\end{equation}

Los exponentes $ \alpha $ son:

\begin{equation}
\alpha_1 = 1.40 \qquad \alpha_2 = 2.90
\end{equation}

In [2]:
#alpha_1 = 1.45
#alpha_2 = 2.91

alpha_1 = 1.453
alpha_2 = 2.911

# Paso 0: Formar matrices $H^{core}$ y de Solapamiento

El primer paso para el calculo Hartree-Fock-Roothaan es calcular la matriz de solapamiento($S_{\mu\nu}$)
y del Hamiltoniano de core ($H^{core}_{\mu\nu}$):

Es fácil verificar que la integral de solpamiento esta dada por:

\begin{equation}
S_{12} = S_{21} = \frac{8\alpha_1^{3/2}\alpha_2^{3/2}}{(\alpha_1+\alpha_2)^3}
\end{equation}

y
\begin{equation}
S_{11} = S_{22} = 1
\end{equation}


In [3]:
S_12 = (8.0*alpha_1**(3./2.)*alpha_2**(3./2.))/(alpha_1+alpha_2)**(3.0)
S_21 = S_12
S_11 = 1
S_22 = S_11

S = np.array = ([[S_11,S_12],[S_21,S_22]])
print S
print np.linalg.inv(S)
evals,evecs=np.linalg.eig(S)
print evals
print evecs
a=np.zeros([2,2]) # inversa de raiz de evals
#eval[0]=np.sqrt(1/a[0][0])
#eval[1]=np.sqrt(1/a[1][1])

a[0][0]=np.sqrt(1/evals[0])
a[1][1]=np.sqrt(1/evals[1])

print a[0][0],a[1][1]
print a
#s=np.empty([2,2]) # matriz de  s minuscula
s= (evecs).dot(a).dot(np.transpose(evecs)) # S-1/2 que lo uso para el paso 3

print s


[[1, 0.8373316183566577], [0.8373316183566577, 1]]
[[ 3.34587187 -2.8016043 ]
 [-2.8016043   3.34587187]]
[ 1.83733162  0.16266838]
[[ 0.70710678 -0.70710678]
 [ 0.70710678  0.70710678]]
0.737744916433 2.47941044836
[[ 0.73774492  0.        ]
 [ 0.          2.47941045]]
[[ 1.60857768 -0.87083277]
 [-0.87083277  1.60857768]]


Los elemento diagonales del Hamiltoniano de core estan dado por:

\begin{equation}
H_{ii} =  0.5\alpha_i^2 - 2\alpha_i 
\end{equation}

y los elementos fuera de la diagonal son:

\begin{equation}
H_{12} = H_{21} = \frac{\alpha_1^{3/2}\alpha_2^{3/2}(4\alpha_1\alpha_2 - 8\alpha_1-8\alpha_2)}{(\alpha_1 + \alpha_2)^3}
\end{equation}


In [4]:
H_11 = 0.5*alpha_1**2 - 2*alpha_1
H_22 = 0.5*alpha_2**2 - 2*alpha_2
H_12 = 4*alpha_1**(3./2.)*alpha_2**(3./2.)*(alpha_1*alpha_2 - 2.0*alpha_1 - 2.0*alpha_2)/(alpha_1+alpha_2)**3.
H_21 = H_12
H = np.array = ([[H_11,H_12],[H_21,H_22]])
print H


[[-1.8503955, -1.8832915267456327], [-1.8832915267456327, -1.5850394999999997]]


Para poder calcular la matriz de Fock, necesitamos calcular las integrales de dos electrones:

# Paso 1: Calcular las integrales de repulsión electrónica (ERI)

Determine las integrales únicas $<\mu\lambda|\nu\sigma>$  de dos electrones para el átomo de He con dos funciones base, y guardelas en un tensor de cuatro indices ERI.



In [5]:
# Integrales de dos electrones:
eri1=5./8.*alpha_1 #<11|11>
eri2=5./8.*alpha_2 #<22|22>
eri3=(alpha_1**4*alpha_2+4*alpha_1**3*alpha_2**2+4*alpha_1**2*alpha_2**3+alpha_1*alpha_2**4)/(alpha_1+alpha_2)**4 #<12|12 >
eri4=20*alpha_1**3*alpha_2**3/(alpha_1+alpha_2)**5 #<11|22>
eri5=16*alpha_1**(9./2.)*alpha_2**(3./2.)/(3*alpha_1+alpha_2)**4*((12*alpha_1+8*alpha_2)/(alpha_1+alpha_2)**2+(9.*alpha_1+alpha_2)/(2.*alpha_1**2))#<11|12>
eri6=16*alpha_2**(9./2.)*alpha_1**(3./2.)/(3*alpha_2+alpha_1)**4*((12*alpha_2+8*alpha_1)/(alpha_2+alpha_1)**2+(9.*alpha_2+alpha_1)/(2.*alpha_2**2))#<22|21>
ERI=np.zeros([2,2,2,2])
ERI[0][0][0][0] = eri1
ERI[0][0][0][1] = eri5
ERI[0][0][1][0] = eri5
ERI[0][0][1][1] = eri4

ERI[0][1][0][0] = eri5
ERI[0][1][0][1] = eri3
ERI[0][1][1][0] = eri4
ERI[0][1][1][1] = eri6

ERI[1][0][0][0] = eri5
ERI[1][0][0][1] = eri4
ERI[1][0][1][0] = eri3
ERI[1][0][1][1] = eri6

ERI[1][1][0][0] = eri4
ERI[1][1][0][1] = eri6
ERI[1][1][1][0] = eri6
ERI[1][1][1][1] = eri2


    # Construción del tensor de dos electrones



# Paso 2: Construir la matriz de transformción

Para poder emepzar con la iteración, debemos pasar a la base ortogonal de las funciones base y así tratar
la ecuación matricial de Hartree-Fock-Roothaan como un problema de valores propios. Procedemos a diagonalizar la matriz de solapamiento y obtener la matriz diagonal s.


\begin{equation}
\Large
\mathbf{s} = \mathbf{U^{\dagger}}\mathbf{S}\mathbf{U}
\end{equation}


Ahora que es una matriz diagonal, le podemos calcular el inverso de la raiz cuadrada: $\quad\mathbf{s^{-1/2}}$


Finalmente construimos la matriz de transformación de base $ \mathbf{S^{-1/2}} $ en 
la base original, utilizando la matriz de vecotres propios $\mathbf{U}$ de 
transformación de base.

\begin{equation}
\Large
\mathbf{S^{-1/2}} = \mathbf{U}\mathbf{s^{-1/2}}\mathbf{U^{\dagger}}
\end{equation}

# Paso 3: Construir la matriz de Fock inicial y encontrar la matriz de coeficientes en la base ortogonal

Ahora podemos formar la matriz de Fock, que en la primer iteración es la matriz del Hamiltoniano
core:



\begin{equation}
\Large
\mathbf{F} = \mathbf{H^{core}}
\end{equation}

Para poder solucionar el problema de valores propios debemos transformar la matriz de Fock
a la base de orbitales ortogonales:



\begin{equation}
\Large
\mathbf{F^{'}} = \mathbf{S^{\dagger -1/2}}\mathbf{F}\mathbf{S^{-1/2}}
\end{equation}


En esta base se cumple que:



\begin{equation}
\Large
\mathbf{F^{'}C^{'}} = \mathbf{C^{'}\epsilon}
\end{equation}




In [6]:
Fprima= np.transpose(s).dot(H).dot(s)
print Fprima

[[-0.71371595 -1.48888739]
 [-1.48888739 -0.22833407]]


In [46]:
evalsfp,evecsfp=np.linalg.eig(Fprima)
print evalsfp
print evecsfp

[-1.97956224  1.03751222]
[[-0.76186558  0.64773516]
 [-0.64773516 -0.76186558]]


# Paso 4: Volver a la base original y formar la matriz de Densidad inicial:

Para obtener los coeficientes en la base original debemos transformar de vuelta, utilizando la matriz 
de transformación simétrica:

\begin{equation}
\Large
\mathbf{C} = \mathbf{S^{-1/2}}\mathbf{C^{'}}
\end{equation}


Con los coeficientes en la base original, es posible caluclar la matriz densidad. Nótese 
que la sum para construir la matriz densidad es sobre los electrones y no sobre las funciones base:


\begin{equation}
\Large
\mathbf{D} = 2\sum_{i=1}^{N/2} C_{\mu i}C_{\nu i} 
\end{equation}


In [59]:
c = (s).dot(evecsfp)
print c


[[-0.66145097  1.70538983]
 [-0.37847481 -1.78958897]]


In [54]:
#D=np.zeros([2,2])
def matden(c): #D espin restricted
    D=np.zeros([2,2]) 
#for u in range(1):
#    for v in range(1):
#        D[u][v]=c[u][0]+c[v][0]
#        print D[u][v]
#D[u][v]= 2*D[u][v]
#print D
    D[0][0]= 2*(c[0][0]*c[0][0]) #D11 
    D[0][1]= 2*(c[0][0]*c[1][0]) #D12
    D[1][0]= 2*(c[1][0]*c[0][0]) #D21
    D[1][1]= 2*(c[1][0]*c[1][0]) #D22
    return D
#matd = matden(c)
#print matd
#D[0][0]= 2*(c[0][0]*c[0][0]) #D11 
#D[0][1]= 2*(c[0][0]*c[1][0]) #D12
#D[1][0]= 2*(c[1][0]*c[0][0]) #D21
#D[1][1]= 2*(c[1][0]*c[1][0]) #D22
#print D

[[ 0.87503477  0.50068506]
 [ 0.50068506  0.28648637]]


# Paso 5: Calcular la energía HF inicial


Ahora podemos calcular la energía Hartree-Fock inicial:

\begin{equation}
\Large
E_{HF} = 0.5\sum_{\mu \nu}^{AO} D_{\mu \nu}(H_{\mu \nu} + F_{\mu \nu})
\end{equation}

In [76]:


matd = matden(c)
D = matd

def energia(D,H,F): #:se usa solo en la primera vez F = Hcore
    DHFT =0.0
    HF = np.zeros([2,2]) 
    DHF = np.zeros([2,2])
#print Fprima
    for u in range(2):
        for v in range(2):
            HF[u][v] = H[u][v]+ F[u][v] # se usa solo en la primera vez F = Hcore
            DHF[u][v] = D[u][v] * HF[u][v] 
            DHFT = DHFT + DHF[u][v]
#            print DHF[u][v]
    DHFT = DHFT*0.5
     
#    print DHFT
    return DHFT

#matd = energia(D,H)
#print matd


In [77]:
#matd = matden(c)
#D = matd

DHFT =0.0
HF = np.zeros([2,2]) 
DHF = np.zeros([2,2])
#print Fprima
for u in range(2):
    for v in range(2):
        HF[u][v] = H[u][v]+ Fe[u][v] # se usa solo en la primera vez F = Hcore
        DHF[u][v] = De[u][v] * HF[u][v] 
        DHFT = DHFT + DHF[u][v]
        print DHF[u][v],DHFT
DHFT = DHFT*0.5
     
print DHFT
#print H   

-3.80960218428 -3.80960218428
-0.89517028234 -4.70477246662
-0.89517028234 -5.59994274896
-0.123059546082 -5.72300229505
-2.86150114752


Con los coeficientes nuevos podemos construir la matriz de Fock inicial, esta vez utliziando las integrales de repulsión electrónica: 

\begin{equation}
\Large
F_{\lambda \sigma} = \sum_{\mu \nu}^{AO} H_{\mu \nu} + D_{\mu \nu}(<\mu\sigma|\nu\lambda> - 0.5<\mu\sigma|\lambda\nu>)
\end{equation}


In [27]:

def Fockn(D):
    DI_ein = np.einsum('uv,usvl',D,ERI)# notacion de einstein
    DII_ein = np.einsum('uv,uslv',D,ERI)

#for u in range(2):
#    for v in range(2):
#        HD[u][v] = H[u][v] +  D[u][v]*((ERI[u][0][v][0]) - 0.5*(ERI[u][0][0][v]))
    HD = H +  (DI_ein) -0.5*(DII_ein)     
#        F11 = F11 + HD[u][v]
    return(HD) 
#   print HD
        
       
#fokn = Fockn(D)
#print fokn
#print F11 
#print FN[0][0]

[[-0.79730172 -0.87930134]
 [-0.87930134 -0.05540369]]


# Paso 5: Calcular la matriz de Fock nueva y repetri los pasos anterioes

Con esta matriz de Fock repetimos los pasos anteriores para obtener una nueva energía electrónica:

In [84]:
#print HD
Fnueva = np.transpose(s).dot(HD).dot(s)
print Fnueva

evalsfn,evecsfn=np.linalg.eig(Fnueva)
#print evalsfp
#print evecsfn
CN = np.empty([2,2])

CN = (s).dot(evecsfn)
c = CN
print CN
#print c
################

matd = matden(c)
print matd
D = matd
#
fokn = Fockn(D)
F = fokn
print F

energ = energia(D,H,F)
print energ




[[ 0.35840319 -1.74755805]
 [-1.74755805  1.71546263]]
[[-0.83554926  1.62718447]
 [-0.1899423  -1.81928387]]
[[ 1.39628514  0.3174123 ]
 [ 0.3174123   0.07215616]]
[[-0.87798861 -0.93692145]
 [-0.93692145 -0.12042202]]
-2.86150114752


# Paso 6: Iterar hasta alcanzar la convergencia:
Finalmente, iteramos hasta alcanzar convergencia.

In [18]:
i = 1
E_thresh = 1e-8